### Word In Microsoft Coco

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/4-1-Word In Microsoft Coco"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df, search_list, target_column, sample_num)\n
    df is dataframe and target_column is its column for external searching_list\n
    sample_num for take sample.
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df, search_list, target_column, sample_num)\n
    df is dataframe and target_column is its column for external searching_list\n
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Microsoft Coco Data

In [6]:
df_coco_question_answers = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/4-0-MS Coco Process/Microsoft_Coco_Question_Answers_Analysis.csv")
df_coco_question_answers

,image_id,question,question_id,multiple_choice_answer
0,262148,where is he looking,262148000,down
1,262148,what are the people in the background doing,262148001,watching
2,262148,what is he on top of,262148002,picnic table
3,393225,what website copyrighted the picture,393225000,foodiebakercom
4,393225,is this a creamy soup,393225001,no
...,...,...,...,...
214349,393212,what is the main color in the photo,393212000,green
214350,393212,what is the meaning of this sign,393212001,go left
214351,393212,what is on the sign,393212002,arrow
214352,393212,does the arrow point left or right,393212003,left


In [7]:
df_coco_captions_instances = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/4-0-MS Coco Process/Microsoft_Coco_Captions_Instances_Analysis.csv")
df_coco_captions_instances

,image_id,caption,license,file_name,coco_url,height,width,flickr_url
0,203564,a bicycle replica with a clock as the front wheel,4,000000203564.jpg,http://images.cocodataset.org/train2017/000000...,400,400,http://farm8.staticflickr.com/7366/9643253026_...
1,203564,the bike has a clock as a tire,4,000000203564.jpg,http://images.cocodataset.org/train2017/000000...,400,400,http://farm8.staticflickr.com/7366/9643253026_...
2,203564,a black metal bicycle with a clock inside the ...,4,000000203564.jpg,http://images.cocodataset.org/train2017/000000...,400,400,http://farm8.staticflickr.com/7366/9643253026_...
3,203564,a bicycle figurine in which the front wheel is...,4,000000203564.jpg,http://images.cocodataset.org/train2017/000000...,400,400,http://farm8.staticflickr.com/7366/9643253026_...
4,203564,a clock with the appearance of the wheel of a ...,4,000000203564.jpg,http://images.cocodataset.org/train2017/000000...,400,400,http://farm8.staticflickr.com/7366/9643253026_...
...,...,...,...,...,...,...,...,...
616616,537802,three teddy bears sit on a fake sled in fake snow,1,000000537802.jpg,http://images.cocodataset.org/val2017/00000053...,480,640,http://farm6.staticflickr.com/5216/5385022107_...
616617,537802,a picture of stuffed animals on a sled in a fa...,1,000000537802.jpg,http://images.cocodataset.org/val2017/00000053...,480,640,http://farm6.staticflickr.com/5216/5385022107_...
616618,537802,three teddy bears sit in a sled in fake snow,1,000000537802.jpg,http://images.cocodataset.org/val2017/00000053...,480,640,http://farm6.staticflickr.com/5216/5385022107_...
616619,537802,three stuffed bears wearing clothes riding on ...,1,000000537802.jpg,http://images.cocodataset.org/val2017/00000053...,480,640,http://farm6.staticflickr.com/5216/5385022107_...


#### Word Search In Coco Data

In [12]:
#df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
#df_word_select

In [13]:
#word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [14]:
#df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
#df_word_raw.reset_index(drop=True, inplace=True)
#df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
#df_word_raw

In [8]:
df_word_raw = pd.read_excel("Turkish_45000_Process.xlsx")
df_word_raw

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [9]:
df_word_raw_word_trans_list = list(set(df_word_raw["word_en_translate"]))
df_word_raw_lemma_trans_list = list(set(df_word_raw["lemma_en_translate"]))

In [10]:
len(df_word_raw_word_trans_list)

818

In [11]:
len(df_word_raw_lemma_trans_list)

486

In [31]:
df_question_answers_search_word_result = word_group_dataframe(df_coco_question_answers, df_word_raw_word_trans_list, "question", 50)
df_question_answers_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_question_answers_search_word_result

,word_en_translate,image_id,questions,que_and_anw_id,answers
0,in the middle,2364313,what in the middle,1719475,beans
1,in the middle,2332440,who is in the middle,337697,a baby elephant
2,in the middle,2391747,who is in the middle,1491577,a boy with a red shirt
3,in the middle,2377578,who is in the middle,365574,the man in the red tie
4,in the middle,2369526,who is in the middle,1825087,little boy
...,...,...,...,...,...
17987,to find,2370075,where would you look to find a reflection,1835712,the mirror
17988,to find,2409292,where can you go in arizona to find a ted s,182055,tempe
17989,to find,2398410,where would a person go to find public parking,1400501,to the left
17990,to find,2390416,what will these people hope to find in the water,1357514,waves


In [32]:
df_question_answers_search_word_result.to_csv("Microsoft_Coco_Question_Answers_Word_Result.csv", index=False)

In [33]:
df_question_answers_search_word_result2 = word_group_dataframe(df_coco_question_answers, df_word_raw_word_trans_list, "multiple_choice_answer", 50)
df_question_answers_search_word_result2.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_question_answers_search_word_result2

,word_en_translate,image_id,questions,que_and_anw_id,answers
0,in the middle,2415083,where is the orange,135731,in the middle
1,in the middle,2373328,where is the big duck,1903693,in the middle
2,in the middle,2373769,where are the carrots,624441,in the middle
3,in the middle,2373917,where is the hot dog and pickle,1916072,in the middle
4,in the middle,2374351,where are the lines in the road,1924266,in the middle
...,...,...,...,...,...
17206,to find,2358095,why do the hats match,509096,to find the rented bike
17207,to find,2343787,what are the giraffes doing,776391,trying to find some shade
17208,to find,2347366,why is the lady picking food,773533,because she wants to find food to buy
17209,to find,2372158,why is the person walking away,1878746,perhaps to find a new place to skateboard


In [34]:
df_question_answers_search_word_result2.to_csv("Microsoft_Coco_Question_Answers_Word_Result2.csv", index=False)

In [35]:
df_question_answers_search_lemma_result = word_group_dataframe(df_coco_question_answers, df_word_raw_lemma_trans_list, "question", 50)
df_question_answers_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_question_answers_search_lemma_result

,lemma_en_translate,image_id,questions,que_and_anw_id,answers
0,dead,2414032,what is dead,1022900,the grass
1,dead,2412069,what is dead,1116811,tree
2,dead,2326071,what is dead,726570,tree branch
3,dead,2410502,what is dead,1110687,leaves
4,dead,2354718,what is dead,383856,sheep
...,...,...,...,...,...
14207,drink,2356300,what drink is that,318609,coffee
14208,drink,2361403,where is the drink,250521,beside the plate
14209,drink,2360965,where is the drink,1652660,in the cup
14210,drink,2403483,where is the drink,1283744,bottom right


In [36]:
df_question_answers_search_lemma_result.to_csv("Microsoft_Coco_Question_Answers_Lemma_Result.csv", index=False)

In [37]:
df_question_answers_search_lemma_result2 = word_group_dataframe(df_coco_question_answers, df_word_raw_lemma_trans_list, "multiple_choice_answer", 50)
df_question_answers_search_lemma_result2.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_question_answers_search_lemma_result2

,lemma_en_translate,image_id,questions,que_and_anw_id,answers
0,dead,61544,what is the condition of the limb,992581,dead
1,dead,2373611,what are the leaves on the ground,1909530,dead
2,dead,2373160,why is the grass brown,1900279,dead
3,dead,2403030,what condition is the grass,1273791,dead
4,dead,2372965,how are the leaves like,1896323,dead
...,...,...,...,...,...
13981,drink,2325424,what is the man holding in his right hand,855084,drink
13982,drink,61520,what is the man carrying,991564,drink
13983,drink,2372119,why is the can opened,1877783,drink
13984,drink,2327355,what is in the boy s left hand,533694,drink


In [38]:
df_question_answers_search_lemma_result2.to_csv("Microsoft_Coco_Question_Answers_Lemma_Result2.csv", index=False)

In [39]:
df_captions_instances_search_word_result = word_group_dataframe(df_coco_captions_instances, df_word_raw_word_trans_list, "caption", 50)
df_captions_instances_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_captions_instances_search_word_result

,word_en_translate,image_id,phrases,region_id,height,width,x_koor,y_koor
0,in the middle,1824,man in the middle,5020001,92,75,370,220
1,in the middle,2335234,net in the middle,3695256,11,23,453,132
2,in the middle,2321022,bus in the middle,4730964,191,60,195,133
3,in the middle,2335054,man in the middle,3703882,132,120,186,112
4,in the middle,2403988,road in the middle,567874,150,250,6,182
...,...,...,...,...,...,...,...,...
22735,to find,2376277,cows are trying to find something to eat on gr...,1693716,39,102,63,246
22736,to find,2369988,snowfall on a field makes it hard to find gras...,2191100,85,493,2,288
22737,to find,2400473,oblong sign telling customers where to find mo...,733030,90,170,2,85
22738,to find,2348549,instructions where to find your life jacket if...,3214722,25,257,111,0


In [40]:
df_captions_instances_search_word_result.to_csv("Microsoft_Coco_Captions_Instances_Word_Result.csv", index=False)

In [41]:
df_captions_instances_search_lemma_result = word_group_dataframe(df_coco_captions_instances, df_word_raw_lemma_trans_list, "caption", 50)
df_captions_instances_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_captions_instances_search_lemma_result

,lemma_en_translate,image_id,phrases,region_id,height,width,x_koor,y_koor
0,dead,2365024,dead bug,2427955,15,19,17,315
1,dead,2387126,dead wood,1773947,102,161,28,349
2,dead,2335986,dead leaf,3659889,28,43,243,412
3,dead,2342954,dead tree,3481126,72,62,428,158
4,dead,2394235,dead tree,1033141,267,116,12,0
...,...,...,...,...,...,...,...,...
18140,drink,2332443,two drink cans,3982934,70,450,23,0
18141,drink,2399986,a sports drink,756557,58,68,2,136
18142,drink,2360170,two drink taps,2659805,125,59,236,102
18143,drink,2379965,a drink cooler,1518899,41,50,382,250


In [42]:
df_captions_instances_search_lemma_result.to_csv("Microsoft_Coco_Captions_Instances_Lemma_Result.csv", index=False)

#### Copy Move And Delete

In [46]:
output_file = glob.glob(f"Microsoft_Coco_*_Result*.csv")
output_file

['Visual_Genome_Question_Answers_Word_Result2.csv',
 'Visual_Genome_Question_Answers_Lemma_Result2.csv']

In [47]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [48]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass